# Gestion des biblothèques

### Installation de pulp

In [7]:
!pip install pulp


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importation des bibliothèques

In [8]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize
import pandas as pd
import ast

# Pré traitement des données

### Importation du dataset

In [9]:
recipes_df = pd.read_csv("../data/raw/RAW_recipes.csv")

### Suppression des colonnes inutiles

In [10]:
recipes_df = recipes_df.drop(columns=['id', 'submitted', 'contributor_id', 'tags', 'description', 'ingredients', 'steps'])
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   name           231636 non-null  object
 1   minutes        231637 non-null  int64 
 2   nutrition      231637 non-null  object
 3   n_steps        231637 non-null  int64 
 4   n_ingredients  231637 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 8.8+ MB


### Suppression des lignes incorrectes

In [11]:
recipes_df.drop_duplicates()
recipes_df.dropna(inplace=True)

### Découpage de la colonne nutrition

In [12]:
recipes_df['nutrition'] = recipes_df['nutrition'].apply(ast.literal_eval)
nutrition_columns = recipes_df['nutrition'].apply(pd.Series)

# Renommer les colonnes
nutrition_columns.columns = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']

# Ajouter les nouvelles colonnes au DataFrame original
recipes_df = pd.concat([recipes_df, nutrition_columns], axis=1)

# Suppression de la colonne nutrition
recipes_df = recipes_df.drop(columns=['nutrition'])

#### Transformation des flottants en entiers

In [13]:
for column in nutrition_columns.columns:
  recipes_df[column] = recipes_df[column].astype(int)

# Programmation par contrainte

### Définition des préférences utilisateurs

In [14]:
user_preferences = {
    "calories": (200, 400), # Plage souhaitée pour les calories
    "total_fat": (0, 30),   # Plage souhaitée pour les matières grasses
    "sugar": (0, 20),       # Plage souhaitée pour le sucre
    "sodium": (0, 5)        # Plage souhaitée pour le sel
}

criteria = ["calories", "total_fat", "sugar", "sodium"]

### Définitions des paramètres

In [15]:
n = len(recipes_df)
max_total_calories = 1000
min_total_protein = 20

### Définition de la satisfaction

#### Création de la fonction

In [16]:
def compute_satisfaction(row, preferences):
    satisfaction = 0

    for criterion, (min_val, max_val) in preferences.items():
        value = row[criterion]

        if min_val <= value <= max_val:
            satisfaction += 1  # Complètement dans la plage
        elif value < min_val:
            satisfaction += 1 - (min_val - value) / min_val  # En dessous de la plage
        elif value > max_val:
            satisfaction += 1 - (value - max_val) / max_val  # Au dessus de la plage

    return satisfaction

#### Calcul du score pour toutes les recettes

In [17]:
recipes_df["satisfaction"] = recipes_df.apply(
    lambda row: compute_satisfaction(row, user_preferences), axis=1
)

recipes_df = recipes_df.reset_index(drop=True)

### Définition du problème

In [18]:
prob = LpProblem("Maximize_User_Satisfaction", LpMaximize)

#### Définition de la variable

In [19]:
X = [LpVariable(f"X{i}", cat="Binary") for i in range(n)]

#### Définition des contraintes

In [20]:
# Maximiser la satisfaction totale
prob += lpSum(recipes_df["satisfaction"][i] * X[i] for i in range(n)), "Total_Satisfaction"

In [21]:
# Contrainte sur la quantité globale de calories
prob += lpSum(recipes_df["calories"][i] * X[i] for i in range(n)) <= max_total_calories, "Max_Calories"

In [22]:
# Contrainte sur la quantité globale de protéine
prob += lpSum(recipes_df["protein"][i] * X[i] for i in range(n)) >= 20, "Max_Protein"

In [23]:
# Limiter aux meilleurs 25 recettes maximum
max_recipes = 25
prob += lpSum(X[i] for i in range(n)) <= max_recipes, "Max_Recipes"

### Résolution du problème

In [24]:
status = prob.solve()

#### Affichage des résultats

In [25]:
if status == 1:
    print("Solution optimale trouvée !")

    # Récupération des recette sélectionnées
    selected_recipes = [i for i in range(n) if X[i].value() == 1]

    # Calcul des taux en calories et protéine
    total_calories = sum(recipes_df["calories"][i] * X[i].value() for i in range(n))
    total_protein = sum(recipes_df["protein"][i] * X[i].value() for i in range(n))

    # Affichage dans la console
    print(f"Nombre de recettes sélectionnées : {len(selected_recipes)}")
    print(f"Calories totales : {total_calories}")
    print(f"Protéines totales : {total_protein}")

    print("Recettes proposées :")
    display(recipes_df.iloc[selected_recipes])

else:
    print("Pas de solution optimale trouvée.")

Solution optimale trouvée !
Nombre de recettes sélectionnées : 25
Calories totales : 1000.0
Protéines totales : 157.0
Recettes proposées :


,name,minutes,n_steps,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates,satisfaction
4506,amaretto tea,5,3,3,0,0,0,0,0,0,0,3.000
4518,amarula coffee,5,1,2,0,0,0,0,0,0,0,3.000
4532,amarula latte,5,2,3,0,0,0,0,0,0,0,3.000
14932,baked potatoes with corn and crema mexicana,70,8,9,200,1,14,4,10,0,15,4.000
34482,camping coffee percolator pot,35,9,3,0,0,0,0,0,0,0,3.000
35728,caramel coconut and almond latte,5,2,5,0,0,0,0,0,0,0,3.000
36063,caravella iced tea,3,4,3,1,0,0,0,0,0,0,3.005
46025,chicken tetrazzini with a twist,35,7,10,198,13,20,4,40,11,2,3.990
53157,ciclon heat storm,1,3,2,0,0,0,1,0,0,0,3.000
59090,country fried ham with red eye gravy,40,8,4,0,0,0,0,0,0,0,3.000
